# Lab 4.4.7: Building a Streamlit Dashboard

**Module:** 4.4 - Containerization & Cloud Deployment  
**Time:** 2 hours  
**Difficulty:** ⭐⭐ (Beginner-Intermediate)

---

## Learning Objectives

By the end of this lab, you will:
- [ ] Build multi-page Streamlit applications
- [ ] Implement chat interfaces with session state
- [ ] Create real-time metric dashboards
- [ ] Add model comparison features
- [ ] Deploy to Streamlit Cloud

---

## Prerequisites

- Basic Python knowledge
- Completed: Lab 4.4.6 (Gradio Demo)
- Ollama running locally (optional, mock mode available)

---

## Real-World Context

**Why Streamlit over Gradio?**

| Feature | Gradio | Streamlit |
|---------|--------|----------|
| Best for | ML demos | Data apps |
| Multi-page | Limited | Native |
| Charts | Basic | Plotly/Altair |
| State mgmt | Limited | Full session state |
| Customization | Themes | Full Python |

**Use Streamlit when you need:**
- Multi-page applications
- Rich data visualizations
- Complex state management
- Internal tools/dashboards

---

## ELI5: What is Streamlit?

> **Imagine you want to build a control panel for a spaceship...**
>
> Gradio is like a ready-made control panel - buttons, sliders, screens all pre-designed. Great for quick demos!
>
> **Streamlit is like a modular control panel kit.** You pick exactly which buttons, screens, and gauges you want, arrange them your way, and can build multi-room control centers (multi-page apps).
>
> **In AI terms:**
> - Streamlit = Build custom dashboards with Python
> - Session state = Remember things between button clicks
> - Caching = Don't reload the model every time

In [ ]:
# Install Streamlit if needed
# !pip install streamlit>=1.30.0 plotly>=5.18.0

import streamlit as st
print(f"Streamlit version: {st.__version__}")

# Note: Streamlit apps can't run directly in notebooks
# We'll write the code to files and run with 'streamlit run'
print("\nStreamlit apps are run with: streamlit run app.py")
print("We'll create the files in this notebook and you can run them locally.")

In [ ]:
# Import our demo utilities
import sys
sys.path.insert(0, '..')

from scripts.demo_utils import (
    StreamingLLMClient,
    MockLLMClient,
    create_streamlit_dashboard,
)

print("Demo utilities loaded!")

---

## Part 1: Your First Streamlit App

Let's start with a simple "Hello World" Streamlit app.

In [ ]:
# Simple Streamlit app
hello_app = '''
"""Simple Streamlit Hello World App."""

import streamlit as st

# Page configuration
st.set_page_config(
    page_title="Hello Streamlit",
    page_icon="",
    layout="centered",
)

# Title and header
st.title("Hello, Streamlit!")
st.markdown("Welcome to your first Streamlit app.")

# User input
name = st.text_input("What's your name?", "World")

# Display greeting
st.write(f"Hello, {name}!")

# A simple button
if st.button("Click me!"):
    st.balloons()
    st.success("You clicked the button!")

# Sidebar
st.sidebar.header("About")
st.sidebar.info("This is a simple Streamlit demo.")
'''

# Save the app
import os
os.makedirs("../app-examples", exist_ok=True)
with open("../app-examples/hello_streamlit.py", "w") as f:
    f.write(hello_app)

print("Saved: ../app-examples/hello_streamlit.py")
print("\nRun with: streamlit run ../app-examples/hello_streamlit.py")
print("\n" + "=" * 60)
print(hello_app)

### What Just Happened?

| Element | Purpose |
|---------|--------|
| `st.set_page_config()` | Configure page title, icon, layout |
| `st.title()` | Large heading |
| `st.text_input()` | Text input field |
| `st.button()` | Clickable button |
| `st.sidebar` | Side panel for navigation/controls |

**Key insight:** Streamlit reruns the entire script on every interaction!

---

## Part 2: Session State

Since Streamlit reruns on every interaction, we need session state to persist data.

In [ ]:
# Session state demo
session_demo = '''
"""Streamlit Session State Demo."""

import streamlit as st

st.title("Session State Demo")

# Initialize session state
if "counter" not in st.session_state:
    st.session_state.counter = 0

if "messages" not in st.session_state:
    st.session_state.messages = []

# Counter example
st.subheader("Counter")
col1, col2 = st.columns(2)

with col1:
    if st.button("Increment"):
        st.session_state.counter += 1

with col2:
    if st.button("Reset"):
        st.session_state.counter = 0

st.metric("Count", st.session_state.counter)

# Message history example
st.markdown("---")
st.subheader("Message History")

new_message = st.text_input("Add a message")
if st.button("Add") and new_message:
    st.session_state.messages.append(new_message)

st.write("Messages:")
for i, msg in enumerate(st.session_state.messages):
    st.write(f"{i+1}. {msg}")

if st.button("Clear All"):
    st.session_state.messages = []
    st.rerun()  # Force refresh
'''

with open("../app-examples/session_demo.py", "w") as f:
    f.write(session_demo)

print("Saved: ../app-examples/session_demo.py")
print("\nKEY CONCEPTS:")
print("  1. st.session_state persists across reruns")
print("  2. Always check 'if key not in st.session_state'")
print("  3. Use st.rerun() to force a refresh")

---

## Part 3: Building a Chat Interface

Let's build a chat interface with Streamlit's native chat components.

In [ ]:
# Chat interface
chat_app = '''
"""Streamlit Chat Interface with LLM."""

import streamlit as st
import time
import requests

st.set_page_config(
    page_title="AI Chat",
    page_icon="",
    layout="wide",
)

st.title("AI Chat")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []


def get_response(prompt: str) -> str:
    """Get response from LLM."""
    try:
        response = requests.post(
            "http://localhost:11434/api/chat",
            json={
                "model": "qwen3:8b",
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
            },
            timeout=60,
        )
        return response.json()["message"]["content"]
    except:
        # Mock response for demo
        return f"[Mock] You said: {prompt}"


# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Chat input
if prompt := st.chat_input("What's on your mind?"):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Get AI response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = get_response(prompt)
        st.markdown(response)
    
    # Add assistant message
    st.session_state.messages.append({"role": "assistant", "content": response})

# Sidebar with options
with st.sidebar:
    st.header("Options")
    if st.button("Clear Chat"):
        st.session_state.messages = []
        st.rerun()
    
    st.markdown("---")
    st.caption(f"Messages: {len(st.session_state.messages)}")
'''

with open("../app-examples/chat_streamlit.py", "w") as f:
    f.write(chat_app)

print("Saved: ../app-examples/chat_streamlit.py")
print("\nRun with: streamlit run ../app-examples/chat_streamlit.py")

---

## Part 4: Multi-Page Applications

Streamlit supports native multi-page applications using a folder structure.

In [ ]:
# Multi-page app structure
print("MULTI-PAGE APP STRUCTURE:")
print("=" * 60)
print("""
my_app/
├── app.py              # Main entry point
├── pages/
│   ├── 1_Chat.py       # Page 1
│   ├── 2_Metrics.py    # Page 2
│   └── 3_Settings.py   # Page 3
└── utils/
    └── helpers.py      # Shared utilities

The number prefix (1_, 2_, 3_) controls page order.
Streamlit automatically creates navigation in the sidebar.
""")

# Create multi-page structure
import os
os.makedirs("../app-examples/multipage/pages", exist_ok=True)

# Main app.py
main_app = '''
"""Multi-page Streamlit App - Main Entry."""

import streamlit as st

st.set_page_config(
    page_title="ML Dashboard",
    page_icon="",
    layout="wide",
)

st.title("ML Model Dashboard")
st.markdown("Welcome! Use the sidebar to navigate between pages.")

# Quick overview
col1, col2, col3 = st.columns(3)

with col1:
    st.info("**Chat**\n\nInteract with the model")

with col2:
    st.info("**Metrics**\n\nView performance data")

with col3:
    st.info("**Settings**\n\nConfigure the model")
'''

with open("../app-examples/multipage/app.py", "w") as f:
    f.write(main_app)

# Page 1: Chat
page_chat = '''
"""Chat Page."""

import streamlit as st

st.title("Chat")

if "messages" not in st.session_state:
    st.session_state.messages = []

for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.write(msg["content"])

if prompt := st.chat_input("Message"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.session_state.messages.append({"role": "assistant", "content": f"Echo: {prompt}"})
    st.rerun()
'''

with open("../app-examples/multipage/pages/1_Chat.py", "w") as f:
    f.write(page_chat)

# Page 2: Metrics
page_metrics = '''
"""Metrics Page."""

import streamlit as st
import pandas as pd
import random

st.title("Metrics")

# Key metrics
col1, col2, col3 = st.columns(3)
col1.metric("Requests", "1,234", "+12%")
col2.metric("Avg Latency", "145ms", "-8ms")
col3.metric("GPU Usage", "67%", "+5%")

# Sample chart
st.subheader("Latency Over Time")
data = pd.DataFrame({
    "Time": range(100),
    "Latency": [100 + random.randint(-20, 20) for _ in range(100)],
})
st.line_chart(data.set_index("Time"))
'''

with open("../app-examples/multipage/pages/2_Metrics.py", "w") as f:
    f.write(page_metrics)

# Page 3: Settings
page_settings = '''
"""Settings Page."""

import streamlit as st

st.title("Settings")

st.subheader("Model Configuration")
model = st.selectbox("Model", ["qwen3:8b", "mistral:7b", "codellama:7b"])
temperature = st.slider("Temperature", 0.0, 2.0, 0.7)
max_tokens = st.number_input("Max Tokens", 100, 4096, 512)

st.markdown("---")
if st.button("Save Settings", type="primary"):
    st.success("Settings saved!")
'''

with open("../app-examples/multipage/pages/3_Settings.py", "w") as f:
    f.write(page_settings)

print("Created multi-page app structure:")
print("  - app-examples/multipage/app.py")
print("  - app-examples/multipage/pages/1_Chat.py")
print("  - app-examples/multipage/pages/2_Metrics.py")
print("  - app-examples/multipage/pages/3_Settings.py")
print("\nRun with: streamlit run ../app-examples/multipage/app.py")

---

## Part 5: Advanced Features - Caching

In [ ]:
# Caching for performance
caching_demo = '''
"""Streamlit Caching Demo."""

import streamlit as st
import time

st.title("Caching Demo")

# ============================================
# @st.cache_data - Cache data computations
# Good for: DataFrames, API responses, calculations
# ============================================

@st.cache_data(ttl=3600)  # Cache for 1 hour
def expensive_computation(n: int) -> int:
    """Simulate an expensive computation."""
    time.sleep(2)  # Simulate slow operation
    return sum(range(n))


st.subheader("@st.cache_data")
n = st.number_input("Number", 1, 1000000, 1000)

start = time.time()
result = expensive_computation(n)
elapsed = time.time() - start

st.write(f"Sum of 1 to {n}: {result:,}")
st.write(f"Time: {elapsed:.3f}s (cached if < 0.1s)")


# ============================================
# @st.cache_resource - Cache resources
# Good for: ML models, database connections
# ============================================

@st.cache_resource
def load_model():
    """Load ML model (cached across all sessions)."""
    time.sleep(3)  # Simulate model loading
    return {"model": "mock_model", "loaded_at": time.time()}


st.markdown("---")
st.subheader("@st.cache_resource")

if st.button("Load Model"):
    with st.spinner("Loading model..."):
        model = load_model()
    st.success(f"Model loaded! (cached: {model})")


# ============================================
# Cache clearing
# ============================================

st.markdown("---")
if st.button("Clear All Caches"):
    st.cache_data.clear()
    st.cache_resource.clear()
    st.success("Caches cleared!")
'''

with open("../app-examples/caching_demo.py", "w") as f:
    f.write(caching_demo)

print("Saved: ../app-examples/caching_demo.py")
print("\nCACHING DECORATORS:")
print("  @st.cache_data    - For data (DataFrame, JSON, calculations)")
print("  @st.cache_resource - For resources (models, connections)")

---

## Part 6: Complete Production Dashboard

Let's create a complete production-ready dashboard.

In [ ]:
# Generate complete dashboard using our utility
mock_client = MockLLMClient()
dashboard_script = create_streamlit_dashboard(
    client=mock_client,
    title="ML Model Dashboard",
)

with open("../app-examples/production_dashboard.py", "w") as f:
    f.write(dashboard_script)

print("Saved: ../app-examples/production_dashboard.py")
print("\nThis includes:")
print("  - Chat page with history")
print("  - Metrics visualization")
print("  - Settings configuration")
print("\nRun with: streamlit run ../app-examples/production_dashboard.py")

---

## Try It Yourself

### Exercise 1: Add GPU Monitoring

Add a GPU monitoring section to the metrics page.

<details>
<summary>Hint</summary>
Use pynvml to get GPU stats, or create mock data for demo.
</details>

In [ ]:
# Your solution here

# Hint: GPU monitoring code
gpu_code = '''
def get_gpu_metrics():
    """Get GPU metrics."""
    try:
        import pynvml
        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        mem = pynvml.nvmlDeviceGetMemoryInfo(handle)
        util = pynvml.nvmlDeviceGetUtilizationRates(handle)
        pynvml.nvmlShutdown()
        return {
            "memory_used": mem.used / 1e9,
            "memory_total": mem.total / 1e9,
            "utilization": util.gpu,
        }
    except:
        return {"memory_used": 45, "memory_total": 128, "utilization": 42}
'''
print(gpu_code)

---

## Common Mistakes

### Mistake 1: Forgetting Session State

```python
# BAD - counter resets on every interaction
counter = 0
if st.button("Click"):
    counter += 1
st.write(counter)  # Always shows 1

# GOOD - use session state
if "counter" not in st.session_state:
    st.session_state.counter = 0
if st.button("Click"):
    st.session_state.counter += 1
st.write(st.session_state.counter)  # Persists
```

---

### Mistake 2: Caching Mutable Objects

```python
# BAD - cached list can be modified
@st.cache_data
def get_data():
    return [1, 2, 3]  # This list can be modified!

# GOOD - return immutable or use deepcopy
@st.cache_data
def get_data():
    return tuple([1, 2, 3])  # Immutable
```

---

### Mistake 3: Slow Operations Without Caching

```python
# BAD - model loads on every interaction
model = load_heavy_model()  # 10 seconds every time!

# GOOD - cache the resource
@st.cache_resource
def load_model():
    return load_heavy_model()

model = load_model()  # Loads once, cached forever
```

---

## Part 7: Deployment to Streamlit Cloud

In [ ]:
# Create deployment files
import os
os.makedirs("../app-examples/.streamlit", exist_ok=True)

# requirements.txt
requirements = '''streamlit>=1.30.0
pandas>=2.0.0
plotly>=5.18.0
requests>=2.31.0
'''

with open("../app-examples/requirements.txt", "w") as f:
    f.write(requirements)

# config.toml
config = '''[theme]
primaryColor = "#76b900"
backgroundColor = "#0e1117"
secondaryBackgroundColor = "#262730"
textColor = "#fafafa"

[server]
maxUploadSize = 50
'''

with open("../app-examples/.streamlit/config.toml", "w") as f:
    f.write(config)

print("Created deployment files!")
print("\nDEPLOYMENT STEPS:")
print("  1. Push to GitHub repository")
print("  2. Go to share.streamlit.io")
print("  3. Connect your GitHub account")
print("  4. Select repository and app.py file")
print("  5. Click Deploy!")

---

## Checkpoint

You've learned:
- Building Streamlit applications
- Managing state with session_state
- Creating chat interfaces
- Multi-page app structure
- Caching for performance
- Deploying to Streamlit Cloud

---

## Challenge (Optional)

Build a complete model evaluation dashboard with:
1. File upload for test data
2. Model selection dropdown
3. Real-time evaluation metrics
4. Confusion matrix visualization
5. Export results to CSV

---

## Further Reading

- [Streamlit Documentation](https://docs.streamlit.io/)
- [Streamlit Cloud](https://streamlit.io/cloud)
- [Streamlit Components](https://streamlit.io/components)
- [Session State Guide](https://docs.streamlit.io/library/api-reference/session-state)

---

## Cleanup

In [ ]:
# List created files
import os

print("Created files:")
for root, dirs, files in os.walk("../app-examples"):
    for f in files:
        path = os.path.join(root, f)
        print(f"  {path}")

print("\nTo run any app:")
print("  streamlit run ../app-examples/<filename>.py")